# Ejercicio 2
## Jorge Pablo Ávila Gómez

## Ejercicio 2: Hive
En este trabajo se van a utilizar ficheros (disponibles en el curso virtual)
descargados de la Kaggle (https://www.kaggle.com/sp1thas/book-depository-dataset) y contienen información sobre libros y sus autores. Son los archivos
authors.csv y dataset.csv, con una serie de campos separados por ; donde su
estructura es la siguiente:

- authors.csv: author_id; author_name.
- datasets.csv: title; author_id; bestsellers-rank; imprint; publication-date;rating-avg; rating-count

Partiendo de este dataset, desarrolla las órdenes de HiveQL que implementen las
siguientes tareas.

#### 1. (2 puntos) Crea las tablas necesarias para almacenar datos. Pueden ser internas o externas en función de los datos que se desee. La decisión de interna o externa debe estar razonada.
#### 2. (1 puntos) Importa los datos en las tablas creadas.
Se van a crear dos tablas externas ya que tenemos los datos en unos archivos en HDFS y vamos a usar la tabla de Hive solo para consultarlos y responder ciertas consultas pero los datos originales permanecerán en sus correspondientes archivos en HDFS.

In [1]:
! mkdir -p hive-tp1
import os
os.chdir("hive-tp1")
! pwd

/media/notebooks/hive-tp1


In [2]:
! hadoop fs -mkdir /user/$(whoami)/datostp1
! hadoop fs -mkdir /user/$(whoami)/datostp1/authors
! hadoop fs -mkdir /user/$(whoami)/datostp1/dataset
! hadoop fs -put ../tp1-book-depository-dataset/authors.csv datostp1/authors/
! hadoop fs -put ../tp1-book-depository-dataset/dataset.csv datostp1/dataset/

mkdir: `/user/root/datostp1': File exists
mkdir: `/user/root/datostp1/authors': File exists
mkdir: `/user/root/datostp1/dataset': File exists
put: `datostp1/authors/authors.csv': File exists
put: `datostp1/dataset/dataset.csv': File exists


#### Crear la base de datos

In [3]:
%%writefile ej2.hql

create database if not exists ej2
Comment 'BD para ejercicio2 de la tp1'
Location '/user/cloudera/ej2'
With dbproperties ('Creada por'='User','Creada el'='31-Oct-2020');

use ej2;

Overwriting ej2.hql


In [4]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f ej2.hql

2020-10-31 12:31:46,802 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
0: jdbc:hive2://localhost:10000/default> create database if not exists ej2
0: jdbc:hive2://localhost:10000/default> Comment 'BD para ejercicio2 de la tp1'
0: jdbc:hive2://localhost:10000/default> Location '/user/cloudera/ej2'
'Creada el'='31-Oct-2020');0000/default> With dbproperties ('Creada por'='User', 
INFO  : Compiling command(queryId=hive_20201031123131_edfbbd71-ea1a-4e4c-989e-11b0e49dec7d): create database if not exists ej2
Comment 'BD para ejercicio2 de la tp1'
Location '/user/cloudera/ej2'
With dbproperties ('Creada por'='User','Creada el'='31-Oct-2020')
INFO

#### Tablas externas

En las dos tablas que se van a crear es importante usar el parametro `TBLPROPERTIES ("skip.header.line.count"="1");` para que nos saltemos la primera líneas de los documentos que es donde se incluye el nombre de cada columna.

Especificamente para la segunda tabla correspodiente al documento dataset.csv he usado el siguiente comando: 

```ALTER TABLE dataset
SET SERDEPROPERTIES ("timestamp.formats"="DD/MM/YYYY");
```

para que pueda leer correctamente el formato de la columna publication_date y se almacene correctamente como tipo timestamp.

In [5]:
%%writefile ej2.hql

create external table authors (
  author_id  INT,   
  author_name  STRING) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' 
LOCATION '/user/root/datostp1/authors'
TBLPROPERTIES ("skip.header.line.count"="1");


Overwriting ej2.hql


In [6]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -f ej2.hql

2020-10-31 12:31:49,236 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2> 
0: jdbc:hive2://localhost:10000/ej2> create external table authors (
0: jdbc:hive2://localhost:10000/ej2>   author_id  INT,   
0: jdbc:hive2://localhost:10000/ej2>   author_name  STRING) 
\;' dbc:hive2://localhost:10000/ej2> ROW FORMAT DELIMITED FIELDS TERMINATED BY ' 
0: jdbc:hive2://localhost:10000/ej2> LOCATION '/user/root/datostp1/authors'
); jdbc:hive2://localhost:10000/ej2> TBLPROPERTIES ("skip.header.line.count"="1" 
INFO  : Compiling command(queryId=hive_20201031123131_9d716162-2216-433d-b15e-6f26ca73e0ec): create external table authors (
  author_id  INT,   
  author

In [7]:
%%writefile ej2.hql

create external table dataset (
  title  STRING,  
  author_id  INT,
  bestseller_rank  INT,
  imprint  STRING,
  publication_date TIMESTAMP,
  rating_avg  FLOAT,
  rating_count INT) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' 
LOCATION '/user/root/datostp1/dataset'
TBLPROPERTIES ("skip.header.line.count"="1");

ALTER TABLE dataset
SET SERDEPROPERTIES ("timestamp.formats" = "dd/MM/yyyy");

Overwriting ej2.hql


In [8]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -f ej2.hql

2020-10-31 12:31:51,706 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2> 
0: jdbc:hive2://localhost:10000/ej2> create external table dataset (
0: jdbc:hive2://localhost:10000/ej2>   title  STRING,  
0: jdbc:hive2://localhost:10000/ej2>   author_id  INT,
0: jdbc:hive2://localhost:10000/ej2>   bestseller_rank  INT,
0: jdbc:hive2://localhost:10000/ej2>   imprint  STRING,
0: jdbc:hive2://localhost:10000/ej2>   publication_date TIMESTAMP,
0: jdbc:hive2://localhost:10000/ej2>   rating_avg  FLOAT,
0: jdbc:hive2://localhost:10000/ej2>   rating_count INT) 
\;' dbc:hive2://localhost:10000/ej2> ROW FORMAT DELIMITED FIELDS TERMINATED BY ' 
0: jdbc:hive2://loca

In [9]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -e 'SELECT * FROM authors LIMIT 10'

2020-10-31 12:31:54,200 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20201031123131_7d7e0de6-5751-4460-9de0-a6d325c515e0): SELECT * FROM authors LIMIT 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:authors.author_id, type:int, comment:null), FieldSchema(name:authors.author_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20201031123131_7d7e0de6-5751-4460-9de0-a6d325c515e0); Time taken: 0.032 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201031123131_7d7e0de6-5751-

In [10]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -e 'SELECT * FROM dataset LIMIT 10'

2020-10-31 12:31:56,603 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20201031123131_e14b6a13-d906-40ff-84b3-bdef81823e67): SELECT * FROM dataset LIMIT 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:dataset.title, type:string, comment:null), FieldSchema(name:dataset.author_id, type:int, comment:null), FieldSchema(name:dataset.bestseller_rank, type:int, comment:null), FieldSchema(name:dataset.imprint, type:string, comment:null), FieldSchema(name:dataset.publication_date, type:timestamp, comment:null), FieldSchema(name:dataset.rating_avg, type:float, comment:null), FieldSchema(name:dat

#### 3. (2 puntos) Crea una vista sobre las tablas creadas. Esta vista tendrá para cada título, el nombre del autor, fecha de publicación, y valoración media.

In [11]:
%%writefile ej2.hql

Create view if not exists vista_ejhive
    AS SELECT d.title, a.author_name, d.publication_date, d.rating_avg
    FROM authors a JOIN dataset d 
    ON a.author_id = d.author_id;

Overwriting ej2.hql


In [12]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -f ej2.hql

2020-10-31 12:31:59,171 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2> 
0: jdbc:hive2://localhost:10000/ej2> Create view if not exists vista_ejhive
lication_date, d.rating_avg0000/ej2>     AS SELECT d.title, a.author_name, d.pub 
0: jdbc:hive2://localhost:10000/ej2>     FROM authors a JOIN dataset d 
0: jdbc:hive2://localhost:10000/ej2>     ON a.author_id = d.author_id;
INFO  : Compiling command(queryId=hive_20201031123232_7ce753cb-09ab-4c93-a843-8c36301deed1): Create view if not exists vista_ejhive
    AS SELECT d.title, a.author_name, d.publication_date, d.rating_avg
    FROM authors a JOIN dataset d 
    ON a.author_id = d.author_id
INFO  : Se

In [13]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -e 'SELECT * FROM vista_ejhive LIMIT 10'

2020-10-31 12:32:01,663 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20201031123232_942a5992-e099-4843-8e4a-e786cb3d92fd): SELECT * FROM vista_ejhive LIMIT 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:vista_ejhive.title, type:string, comment:null), FieldSchema(name:vista_ejhive.author_name, type:string, comment:null), FieldSchema(name:vista_ejhive.publication_date, type:timestamp, comment:null), FieldSchema(name:vista_ejhive.rating_avg, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20201031123232_942a5992-e099-4843-8e4a-e786cb3d92fd)

#### 4. (3 puntos) Crea las consultas de Hive necesarias para responder las siguientes cuestiones:

- ¿Cuál es el título del libro con mejor valoración media de la autora Ana Maria Spagna?

In [14]:
%%writefile ej2.hql

WITH t1 AS (
SELECT title, rating_avg
FROM vista_ejhive 
WHERE author_name = 'Ana Maria Spagna'
ORDER BY rating_avg DESC
LIMIT 1
)
SELECT title 
FROM t1;

Overwriting ej2.hql


In [15]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -f ej2.hql

2020-10-31 12:32:16,850 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2> 
0: jdbc:hive2://localhost:10000/ej2> WITH t1 AS (
0: jdbc:hive2://localhost:10000/ej2> SELECT title, rating_avg
0: jdbc:hive2://localhost:10000/ej2> FROM vista_ejhive 
0: jdbc:hive2://localhost:10000/ej2> WHERE author_name = 'Ana Maria Spagna'
0: jdbc:hive2://localhost:10000/ej2> ORDER BY rating_avg DESC
0: jdbc:hive2://localhost:10000/ej2> LIMIT 1
0: jdbc:hive2://localhost:10000/ej2> )
0: jdbc:hive2://localhost:10000/ej2> SELECT title 
0: jdbc:hive2://localhost:10000/ej2> FROM t1;
INFO  : Compiling command(queryId=hive_20201031123232_24679d83-2bc7-47a4-929d-2621aa3e5c75): WI

- ¿Cuáles son los cinco autores que han escrito más libros?

In [16]:
%%writefile ej2.hql

SELECT author_name, COUNT(title) n_books
FROM vista_ejhive 
GROUP BY author_name
ORDER BY n_books DESC 
LIMIT 5

Overwriting ej2.hql


In [17]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -f ej2.hql

2020-10-31 12:32:36,136 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2> 
0: jdbc:hive2://localhost:10000/ej2> SELECT author_name, COUNT(title) n_books
0: jdbc:hive2://localhost:10000/ej2> FROM vista_ejhive 
0: jdbc:hive2://localhost:10000/ej2> GROUP BY author_name
0: jdbc:hive2://localhost:10000/ej2> ORDER BY n_books DESC 
0: jdbc:hive2://localhost:10000/ej2> LIMIT 5
0: jdbc:hive2://localhost:10000/ej2> INFO  : Compiling command(queryId=hive_20201031123232_2339d90a-53f9-4fb0-a2b2-b16f23c936ca): SELECT author_name, COUNT(title) n_books
FROM vista_ejhive 
GROUP BY author_name
ORDER BY n_books DESC 
LIMIT 5
INFO  : Semantic Analysis Completed
INFO  :

- ¿Cuáles son los dos libros más recientes de la autora Ana Maria Spagna?

In [18]:
%%writefile ej2.hql

SELECT publication_date, title
FROM vista_ejhive 
WHERE author_name = 'Ana Maria Spagna'
ORDER BY publication_date DESC
LIMIT 2

Overwriting ej2.hql


In [19]:
! beeline -u "jdbc:hive2://localhost:10000/ej2" -f ej2.hql

2020-10-31 12:33:11,204 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/ej2
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/ej2> 
0: jdbc:hive2://localhost:10000/ej2> SELECT publication_date, title
0: jdbc:hive2://localhost:10000/ej2> FROM vista_ejhive 
0: jdbc:hive2://localhost:10000/ej2> WHERE author_name = 'Ana Maria Spagna'
0: jdbc:hive2://localhost:10000/ej2> ORDER BY publication_date DESC
0: jdbc:hive2://localhost:10000/ej2> LIMIT 2
0: jdbc:hive2://localhost:10000/ej2> INFO  : Compiling command(queryId=hive_20201031123333_e75499da-e56c-49d8-8bb5-1508ae513aa6): SELECT publication_date, title
FROM vista_ejhive 
WHERE author_name = 'Ana Maria Spagna'
ORDER BY publication_date DESC
LIMIT 2
INFO  : Sem